# Differential expression analysis with edgeR

In this notebook, we will perform differential expression analysis to test for changes in expression between healthy (PBMMC) and leukemia (ETV6-RUNX1) samples for cells of the same 'type' present in both condition.

## Setup

In [ ]:
import decoupler as dc
import pertpy as pt
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
adata = sc.read_h5ad('../Data/Caron_clustered.PBMMCandETV6RUNX1.h5ad')

In [ ]:
adata

In [ ]:
sc.pl.embedding(adata, "X_umap_corrected", color=["label"], legend_loc="on data")

In [ ]:
adata.obs[["label", "SampleName"]].value_counts()

## Create pseudobulk

We want to create an aggregate count (pseudobulk) for each sample and cell type (label) combinations. However, we watn to remove pseudobulk 'samples' with very low number of cells (e.g. <20).

In [ ]:
pdata = dc.get_pseudobulk(
    adata, sample_col="SampleName", groups_col="label", mode="sum", min_cells=20
)

In [ ]:
pdata

In [ ]:
pdata.obs

In [ ]:
pdata.X

## Differential expression analysis

We will now run the differential expression analysis for one of the cell type - B (c1) - to show the steps involved in the analysis.

In [ ]:
def filter_by_expression(adata, group):
    """
    Filter lowly expressed genes using edgeR filterByExpr function.

    :adata: anndata object
    :group: column name in the anndata obs layer containing sample grouping
    
    :return: anndata object with lowly expressed genes removed
    """
    from rpy2.robjects.packages import importr
    import rpy2.robjects as ro
    import rpy2.robjects.numpy2ri
    
    rpy2.robjects.numpy2ri.activate()
    edger = importr("edgeR")
    keep = edger.filterByExpr(adata.X.T, ro.FactorVector(adata.obs[group]))
    return adata[:, list(keep)]

In [ ]:
b_c1_pdata = pdata[pdata.obs['label'] == "B (c1)"].copy()
b_c1_pdata

In [ ]:
b_c1_pdata.obs

We first filter for lowly expressed genes using the helper function we set up previously.

In [ ]:
b_c1_pdata = filter_by_expression(b_c1_pdata, 'SampleGroup')

In [ ]:
b_c1_pdata

We now set up the edgeR object and specify the design of our differential expression analysis. In this case, since we want to compare between the condition, we use `~SampleGroup`. We then fit the linear model. 

In [ ]:
b_c1_edgr = pt.tl.EdgeR(b_c1_pdata, design="~SampleGroup")

In [ ]:
b_c1_edgr.fit(robust=True)

In [ ]:
b_c1_pdata.layers['counts'] = b_c1_pdata.X.copy()
sc.pp.normalize_total(b_c1_pdata)
sc.pp.log1p(b_c1_pdata)
sc.pp.scale(b_c1_pdata, max_value=10)
sc.tl.pca(b_c1_pdata)
sc.pl.pca(b_c1_pdata, color='SampleGroup', palette={'PBMMC': 'tomato', 'ETV6-RUNX1': 'steelblue'}, size=100)

To get the result of the differential expression analysis, we use the `test_contrast` function and pass the comparison we want to perform as it's argument. We can then visualise the result in a volcano plot.

In [ ]:
b_c1_res_df = b_c1_edgr.test_contrasts(b_c1_edgr.contrast(column="SampleGroup", baseline="ETV6-RUNX1", group_to_compare="PBMMC"))
b_c1_res_df.index = b_c1_res_df["variable"]

In [ ]:
b_c1_res_df

In [ ]:
b_c1_edgr.plot_volcano(b_c1_res_df, pval_thresh=0.05, log2fc_thresh=0)

### Optional: Gene set analysis

Now that we have the differentially expressed gene lists, we can use this to identify genes which changes between conditions. While we can do this manually, we can also use further downstream analysis, such as gene set analysis, to 'summarise' the differentially expressed genes into gene sets/modules which are differentially expressed.

In [ ]:
msigdb = dc.get_resource('MSigDB')
msigdb

In [ ]:
# Filter by hallmark
msigdb = msigdb[msigdb['collection']=='hallmark']

# Remove duplicated entries
msigdb = msigdb[~msigdb.duplicated(['geneset', 'genesymbol'])]

# Rename
msigdb.loc[:, 'geneset'] = [name.split('HALLMARK_')[1] for name in msigdb['geneset']]

msigdb

In [ ]:
# Run ora
enr_pvals = dc.get_ora_df(
    df=b_c1_res_df[(b_c1_res_df['adj_p_value'] < 0.05) & (b_c1_res_df['log_fc'] > 0)].index,
    net=msigdb,
    source='geneset',
    target='genesymbol'
)

enr_pvals.head()

In [ ]:
dc.plot_dotplot(
    enr_pvals.sort_values('Combined score', ascending=False).head(15),
    x='Combined score',
    y='Term',
    s='Odds ratio',
    c='FDR p-value',
    scale=0.5,
    figsize=(3, 9)
)

## Putting it all together

Now that we have laid out the process of differential expression analysis for one of the cell type, we can repeat the same process for the remaining cell types in our dataset.

For convenience, we create a function that loops over all the cell types and perform the same analysis as above.

In [ ]:
def pseudo_bulk_dge(adata, label, design, group, column, baseline, group_to_compare):
    """
    Perform pseudobulk differential expression analysis across each cell types within labels.

    :adata: anndata object
    :label: column name in the anndata obs layer containing cell grouping
    :design: a formula to be used to construct design matrix
    :group: column name in the anndata obs layer containing sample grouping
    :column: column name in the design formular argument to specify the comparison to perform
    :baseline: label present in the column argument to set as baseline for comparison
    :group_to_compare: label present in the column argument to compare against the baseline
    
    :return: a dictionary of dgeResults object containing the edgeR object and de_result table for each cell types within labels
    """
    from dataclasses import dataclass

    @dataclass
    class dgeResults:
        edger_object: pt.tools.EdgeR
        de_result: pd.core.frame.DataFrame

        def plot_volcano(self, **kwargs):
            self.edger_object.plot_volcano(self.de_result, **kwargs)
    
    results = {}
    for l in adata.obs[label].unique():
        try:
            subset = adata[adata.obs[label] == l]
            subset = filter_by_expression(subset, group)
            edgr = pt.tl.EdgeR(subset, design=design)
            edgr.fit(robust=True)
            result = edgr.test_contrasts(edgr.contrast(column=column, baseline=baseline, group_to_compare=group_to_compare))
            result.index = result["variable"]
            results[l] = dgeResults(edgr, result)
        except Exception as e:
            print(f"Invalid comparison for {l}: {e}")
    return results

In [ ]:
de_results = pseudo_bulk_dge(pdata, 'label', '~SampleGroup', 'SampleGroup', 'SampleGroup', 'ETV6-RUNX1', 'PBMMC')

In [ ]:
def decide_tests_per_label(de_results, threshold):
    """
    Summarise result of the differential expression analysis.

    :de_results: a dictionary of dgeResults object containing the edgeR object and de_result table for each cell types within labels
    :threshold: FDR adjusted p-value threshold to consider genes as significant
    
    :return: a dataframe containing the number of genes of each DE status (column) in each label (row)
    """
    
    def de_dir(row):
        if row['adj_p_value'] < threshold:
            return 1 if row['log_fc'] > 0 else -1
        else:
            return 0
    return pd.DataFrame({label: res.de_result.apply(de_dir, axis=1) for label, res in de_results.items()})

In [ ]:
is_de = decide_tests_per_label(de_results, 0.05)

In [ ]:
is_de.apply(lambda x:x.value_counts()).T

Let's confirm that we get the same result as before in the B (c1) cells.

In [ ]:
de_results["B (c1)"].de_result

In [ ]:
de_results["B (c1)"].plot_volcano(pval_thresh=0.05, log2fc_thresh=0)

From the volcano plot, we can see that HTR1F gene is highly upregulated in B cells. Let's look at the expression of this genes across all the other cell types.

In [ ]:
pdata.layers["counts"] = pdata.X.copy()
sc.pp.normalize_total(pdata)
sc.pp.log1p(pdata)

In [ ]:
plot_data = pdata.obs.copy()
plot_data["logcounts"] = pdata[:,"HTR1F"].X.flatten().tolist()
sns.catplot(data=plot_data, x="SampleGroup", y="logcounts", 
            palette=['tomato', 'steelblue'], col="label", col_wrap=5,
            height=2, width=3)

We can see that this gene is also upregulated in leukemic cells in other B cell types.

----

### Optional exercises

1. Explore the DE result for other cell types and run some gene set analysis.
2. Create a function to automatically perform gene set analysis for each label in the cell types.
